In [ ]:
# !pip install tensorflow-datasets
!pip install -U tfds-nightly
# !pip install tensorflow-gpu

In [ ]:
# !pip install -U --pre tensorflow=="2.1"
!pip install tf_slim

In [ ]:
!pip install pycocotools

In [ ]:
# !pip install tensorflow==2.1


In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display



In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd models/research
pip install .

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
import tensorflow_datasets as tfds


In [ ]:
# ds = tfds.load('kitti', split='train',data_dir='/content/tensorflow_datasets', shuffle_files=True)
# assert isinstance(ds, tf.data.Dataset)
# print(ds)

In [ ]:
# !gcloud auth login application-default



In [ ]:
!gcloud config set account application-default

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# !gcloud auth login dissertation-0




In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'kitti',
    split=['train', 'test'],
#     shuffle_files=True,
    # as_supervised=True,
#     try_gcs=True,
    with_info=True,
    download=False,
#     data_dir='./tensorflow_datasets'
    data_dir="gs://kitti-dataset-1"
)

# ds = tfds.load('kitti', split='train',data_dir='/content/tensorflow_datasets')


In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [ ]:
# model_name = 'faster_rcnn_resnet101_kitti_2018_01_28'
model_name = 'faster_rcnn_inception_v2_coco_2018_01_28'
# model_name = 'faster_rcnn_nas_lowproposals_coco_2018_01_28'
detection_model = load_model(model_name)


In [ ]:
# List of the strings that is used to add correct label for each box.

# PATH_TO_LABELS = 'models/research/object_detection/data/kitti_label_map.pbtxt'
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


In [ ]:
# ds_train =ds 

In [ ]:
def normalize_img(object):
  print(object['objects'])
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(object['image'], tf.float32) / 255., object['objects']['bbox']



In [ ]:
# ds_train = ds_train.map(
#     normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
# ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
# ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
ds_train

In [ ]:
# ds_test = ds_test.map(
#     normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
print(detection_model.inputs)



In [ ]:
detection_model.output_dtypes

In [ ]:
detection_model.output_shapes

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def process_image(image):
  

    image_np = np.copy(image) #TODO: make more efficent 
    
    output_dict = run_inference_for_single_image(detection_model, image_np)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=1)
    return image_np

In [ ]:
def show_inference(model, tensor):
  
  
    image_np = np.array(tensor)
    image =process_image(image_np)
    display(Image.fromarray(image))
    return

In [ ]:
# for example in ds_test.take(3):  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
#     print(list(example.keys()))
#     image = example["image"]
#     objects = example["objects"]
  
# #     print(objects['location'])
#     show_inference(detection_model, image)




In [ ]:
ds_info

In [ ]:
#Evaluation Steps


from object_detection import eval_util
from object_detection.core import standard_fields as fields
from object_detection.metrics import coco_evaluation
from object_detection.protos import eval_pb2
from object_detection.utils import test_case
from object_detection.utils import tf_version

def test_get_eval_metric_ops_for_coco_detections(eval_dict, batch_size=1,
                                               max_gt_boxes=None,
                                               scale_to_absolute=False):
    eval_config = eval_pb2.EvalConfig()
    eval_config.metrics_set.extend(['pascal_voc_detection_metrics'])
    categories = list(category_index.values())

    metric_ops = eval_util.get_eval_metric_ops_for_evaluators(
        eval_config, categories, eval_dict)
    _, update_op = metric_ops['DetectionBoxes_Precision/mAP']

    with self.test_session() as sess:
        metrics = {}
        for key, (value_op, _) in six.iteritems(metric_ops):
            metrics[key] = value_op
        sess.run(update_op)
        metrics = sess.run(metrics)
        print(metrics)
  

In [ ]:
#TODO: Complete Evaluation steps

input_data_fields = fields.InputDataFields
detection_fields = fields.DetectionResultFields

for example in ds_test.take(1):  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
    print(list(example.keys()))
    image = example["image"]
    objects = example["objects"]
  
#     print( objects)
    print(objects['location'])
#     show_inference(detection_model, image)
    image_np = np.array(image)
    # Actual detection.
    output_dict = run_inference_for_single_image(detection_model, image_np)
    
#       return eval_util.result_dict_for_single_example(
#           image, key, detections, groundtruth,
#           scale_to_absolute=scale_to_absolute)


#     batch_size = 100
#     key = tf.constant([str(i) for i in range(batch_size)])

    key = tf.constant('image1')
    
    groundtruth_boxes = objects['bbox']
    groundtruth_classes = objects['type']
    groundtruth = {
        input_data_fields.groundtruth_boxes: groundtruth_boxes,
        input_data_fields.groundtruth_classes: groundtruth_classes,
      
    }
    
#     num_detections = tf.convert_to_tensor([len(output_dict['detection_classes'])])
    batch_size = 375
    num_detection = tf.ones([batch_size])
    detections = {
        detection_fields.detection_boxes:  output_dict['detection_boxes'],
        detection_fields.detection_scores: output_dict['detection_scores'],
        detection_fields.detection_classes: output_dict['detection_classes'],
        detection_fields.num_detections: num_detections
       
    }
    result_dict = eval_util.result_dict_for_single_example(image, key,detections, groundtruth)
#     test_get_eval_metric_ops_for_coco_detections(output_dict)

In [ ]:
!pip install moviepy

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [56]:
!ls -sh accident_compilation.mp4

110M accident_compilation.mp4


In [ ]:
from moviepy.editor import VideoFileClip
import proglog
proglog.notebook()

write_output = 'output1.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first n seconds
# clip1 = VideoFileClip("test.mp4").subclip(0,1)

# clip1 = VideoFileClip("test.mp4")

clip1 = VideoFileClip("accident_compilation.mp4")


white_clip = clip1.fl_image(process_image) 

white_clip.write_videofile(write_output, audio=False, verbose=False)

clip1.close()


Moviepy - Building video output1.mp4.
Moviepy - Writing video output1.mp4



In [ ]:
!ls